In [1]:
# pip install matplotlib

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('D:/Hyunho Shin - DTU/Autumn-2022/Computational Tools for Data Science/project/recommenderrrrr/df_with_user_id.csv', encoding='latin-1')
len(df)

34376

# Data preprocessing - creating user, movie id

In [3]:
# user_list = df['username'].tolist()
# print('including duplicated:{}'.format(len(user_list)))

# user_list_c = set(user_list)
# user_list_c = list(user_list_c)

# print('excluding duplicated:{}'.format(len(user_list_c)))

# dict_user = {}

# for i in range(len(user_list_c)):
#   dict_user[user_list_c[i]] = i+1
# print(dict_user)

In [4]:
# df_c = df.copy()
# df_c['user_id'] = ""

# user_id = []
# for i in range(len(df_c)):
#   user_id.append(dict_user[df_c.iloc[i]['username']])
# print(user_id)
# df_c['user_id'] = user_id
# # df_c

In [5]:
# # drop duplicated rows
# df_ct = df_c.copy()
# df_ct = df_ct.drop_duplicates(['user_id','movie_id'])
# df_ct = df_ct.reset_index()

In [6]:
# movie_list_tmp = df_ct['original_title'].tolist()

# # print(len(movie_list_tmp))
# movie_list = set(movie_list_tmp)
# movie_list = list(movie_list)
# # print(len(movie_list))

# dict_movie = {}

# for i in range(len(movie_list)):
#   dict_movie[movie_list[i]] = i+1
# print(dict_movie)

In [7]:
# df_ct_c = df_ct.copy()
# df_ct_c['new_movie_id'] = ""

# movie_id = []
# for i in range(len(df_ct_c)):
#   movie_id.append(dict_movie[df_ct_c.iloc[i]['original_title']])
# print(movie_id)
# len(movie_id)
# # df_c['user_id'] = user_id

In [8]:
# df_ct_c['new_movie_id'] = movie_id
# # df_ct_c

In [9]:
# # remove original movie_id column and rename new movie id as movie_id
# df_ct_c = df_ct_c.drop(columns = ['movie_id'])
# df_ct_c = df_ct_c.rename(columns={'new_movie_id':'movie_id'})
# # df_ct_c

In [10]:
# df_ct_c.to_csv('/content/drive/MyDrive/DTU/Computational Tools for Data Science/project/dataframeset/df_with_user_id.csv')

# Recommender

In [11]:
df_cp = df.copy()

In [12]:
df_cp = df_cp[['genres', 'original_title',
       'overview', 'runtime', 'vote_average', 'username', 'rating', 'total',
       'date', 'review', 'cast', 'crew', 'user_id', 'movie_id']]

In [13]:
in_use = df_cp[['user_id','movie_id','rating']]
in_use.rating.replace({-1:np.nan},regex=True, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_11276\2618185387.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_use.rating.replace({-1:np.nan},regex=True, inplace=True)


In [14]:
len(in_use)

34376

In [15]:
piv = in_use.pivot_table(index=['user_id'],columns=['movie_id'],values='rating')

piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)),axis=1)
piv_norm.index
# piv_norm

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            23239, 23240, 23241, 23242, 23243, 23244, 23245, 23246, 23247,
            23248],
           dtype='int64', name='user_id', length=23248)

In [16]:
import scipy as sp
piv_norm.fillna(0,inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:,(piv_norm!=0).any(axis=0)]

piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

item_sim_df = pd.DataFrame(item_similarity, index = piv_norm.index, columns=piv_norm.index)
user_sim_df = pd.DataFrame(user_similarity, index = piv_norm.columns,columns=piv_norm.columns)

In [17]:
item_sim_df.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,740,741,742,743,744,745,746,747,748,749
movie_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.000000,0.000000,0.000000,0.000000,-0.001108,0.000000,0.000000,0.00000,0.004533,...,0.000000,0.000000,0.000000,0.000000,-0.005806,0.013618,-0.030346,0.078645,0.004354,0.000000
2,0.0,1.000000,0.023009,-0.005062,0.010188,-0.001227,0.000000,-0.001239,0.00000,0.014298,...,0.000000,-0.016652,0.000000,-0.002117,0.000000,0.000000,0.011816,0.000000,0.000000,-0.002437
3,0.0,0.023009,1.000000,0.000516,0.003513,-0.001278,0.001369,-0.001291,-0.00609,0.030477,...,-0.008688,-0.035496,-0.009487,-0.002206,0.000000,-0.000825,0.018699,0.000000,0.000000,-0.010401
4,0.0,-0.005062,0.000516,1.000000,-0.002099,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.010188,0.003513,-0.002099,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000710


In [18]:
user_sim_df.head()

user_id,20,24,45,46,48,61,66,67,76,82,...,23159,23173,23174,23176,23178,23179,23200,23203,23216,23232
user_id,,,,,,,,,,,,,,,,,,,,,
20,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
24,0.0,1.000000,0.000000,0.0,0.021077,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.045566,-0.04572,0.000000,0.065124
45,0.0,0.000000,1.000000,0.0,0.004307,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,-0.010401,0.00000,-0.024352,0.045609
46,0.0,0.000000,0.000000,1.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
48,0.0,0.021077,0.004307,0.0,1.000000,0.0,0.085553,0.0,0.01134,0.012464,...,0.0,0.0,0.0,-0.018815,0.0,0.0,-0.012901,0.00000,-0.005156,0.089038


In [19]:
def top_movies(movie_id):
  cnt = 1
  print("Similar movies to {}:\n".format(movie_id))
  result = item_sim_df.loc[~item_sim_df.index.isin([movie_id]),movie_id].sort_values(ascending=False)[:10]

  for id, score in result.items():
    print('No. {}: {}({:.2f})'.format(cnt,id,score))
    cnt+=1

In [20]:
top_movies(245)

Similar movies to 245:

No. 1: 321(0.10)
No. 2: 262(0.08)
No. 3: 525(0.06)
No. 4: 208(0.06)
No. 5: 131(0.03)
No. 6: 251(0.03)
No. 7: 396(0.03)
No. 8: 330(0.02)
No. 9: 607(0.01)
No. 10: 123(0.01)


C:\Users\user\AppData\Local\Temp\ipykernel_11276\2901470906.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  result = item_sim_df.loc[~item_sim_df.index.isin([movie_id]),movie_id].sort_values(ascending=False)[:10]


In [21]:
import operator
def similar_user_recom(user):
  if user not in piv_norm.columns:
    return('No data available on user {}'.format(user))
  
  # get 10 users by the highest similarity
  sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
  best = []
  most_common={}

  for i in sim_users:
    # Get the list of items that 10 users with high similarity gave high ratings.
    # However, be careful that the item you are recommending must be an item that has not been evaluated by the target user.
    result_sorted = piv_norm.loc[:,i][(piv_norm.loc[:,user]==0)].sort_values(ascending=False)
    best.append(result_sorted.index[:5].tolist())
  for j in range(len(best)):
    for k in best[j]:
      if k in most_common:
        most_common[k]+=1
      else:
        most_common[k]=1
  sorted_list = sorted(most_common.items(),key=operator.itemgetter(1),reverse=True)
  return sorted_list[:5]

In [22]:
similar_user_recom(48) # (movie_id, the number of people who have highest user similarity who rated "movie_id"?)

[(499, 7), (1, 6), (500, 6), (498, 4), (508, 3)]

In [23]:
piv_norm

user_id,20,24,45,46,48,61,66,67,76,82,...,23159,23173,23174,23176,23178,23179,23200,23203,23216,23232
movie_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.213501,0.0,0.583333,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.070644,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,-0.072214,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.070644,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
746,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
747,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
def similar_user_recom_100(user):
  if user not in piv_norm.columns:
    return('No data available on user {}'.format(user))
  
  # get 10 users by the highest similarity
  sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:101]
  best = []
  most_common={}

  for i in sim_users:
    # Get the list of items that 10 users with high similarity gave high ratings.
    # However, be careful that the item you are recommending must be an item that has not been evaluated by the target user.
    result_sorted = piv_norm.loc[:,i][(piv_norm.loc[:,user]==0)].sort_values(ascending=False)
    best.append(result_sorted.index[:10].tolist())
  for j in range(len(best)):
    for k in best[j]:
      if k in most_common:
        most_common[k]+=1
      else:
        most_common[k]=1
  sorted_list = sorted(most_common.items(),key=operator.itemgetter(1),reverse=True)
  return sorted_list[:5]

In [25]:
similar_user_recom_100(20)

[(495, 92), (496, 91), (497, 89), (498, 85), (494, 77)]

In [26]:
df

,Unnamed: 0.1,index,Unnamed: 0,genres,original_title,overview,runtime,vote_average,username,rating,total,date,review,cast,crew,user_id,movie_id
0,0,0,0,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 878, ""...",10 Cloverfield Lane,"After a car accident, Michelle awakens to find...",103.0,6.8,Imme-van-Gorp,7,123,30 January 2019,This movie is full of suspense. It makes you g...,"[{""cast_id"": 2, ""character"": ""Michelle"", ""cred...","[{""credit_id"": ""57627624c3a3680682000872"", ""de...",5764,370
1,1,1,1,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 878, ""...",10 Cloverfield Lane,"After a car accident, Michelle awakens to find...",103.0,6.8,sonofocelot-1,5,500,10 May 2016,I'll leave this review fairly concise. <br/><b...,"[{""cast_id"": 2, ""character"": ""Michelle"", ""cred...","[{""credit_id"": ""57627624c3a3680682000872"", ""de...",6791,370
2,2,2,2,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 878, ""...",10 Cloverfield Lane,"After a car accident, Michelle awakens to find...",103.0,6.8,mhodaee,5,143,4 August 2017,I give the 5/10 out of the credit I owe to the...,"[{""cast_id"": 2, ""character"": ""Michelle"", ""cred...","[{""credit_id"": ""57627624c3a3680682000872"", ""de...",7304,370
3,3,3,3,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 878, ""...",10 Cloverfield Lane,"After a car accident, Michelle awakens to find...",103.0,6.8,fil-nik09,5,100,5 October 2016,"First of all, I must say that I was expecting ...","[{""cast_id"": 2, ""character"": ""Michelle"", ""cred...","[{""credit_id"": ""57627624c3a3680682000872"", ""de...",9809,370
4,4,4,4,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 878, ""...",10 Cloverfield Lane,"After a car accident, Michelle awakens to find...",103.0,6.8,DVR_Brale,7,56,27 July 2016,I've always loved movies with strong atmospher...,"[{""cast_id"": 2, ""character"": ""Michelle"", ""cred...","[{""credit_id"": ""57627624c3a3680682000872"", ""de...",770,370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34371,34371,34399,37642,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...",Zulu,"As a child, Ali Neuman narrowly escaped being ...",110.0,6.7,jckruize,8,4,22 October 2001,"Expert mix of elements: exotic locale, strong ...","[{""cast_id"": 2, ""character"": ""Brian Epkeen"", ""...","[{""credit_id"": ""553f81bec3a3685c650033a2"", ""de...",18703,334
34372,34372,34400,37645,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...",Zulu,"As a child, Ali Neuman narrowly escaped being ...",110.0,6.7,baroncoco,10,10,10 September 2019,This is one of the great stories of desperate ...,"[{""cast_id"": 2, ""character"": ""Brian Epkeen"", ""...","[{""credit_id"": ""553f81bec3a3685c650033a2"", ""de...",9641,334
34373,34373,34401,37646,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...",Zulu,"As a child, Ali Neuman narrowly escaped being ...",110.0,6.7,Tweekums,10,10,9 September 2010,This is one of the great British war movies de...,"[{""cast_id"": 2, ""character"": ""Brian Epkeen"", ""...","[{""credit_id"": ""553f81bec3a3685c650033a2"", ""de...",16173,334
34374,34374,34402,37647,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...",Zulu,"As a child, Ali Neuman narrowly escaped being ...",110.0,6.7,steve-hosking,9,10,24 August 2010,Wow. As a History undergraduate it is indescri...,"[{""cast_id"": 2, ""character"": ""Brian Epkeen"", ""...","[{""credit_id"": ""553f81bec3a3685c650033a2"", ""de...",16209,334


In [27]:
df['movie_id'].value_counts()

283    95
638    93
645    90
643    85
138    50
       ..
640    33
83     32
681    31
295    31
13     29
Name: movie_id, Length: 749, dtype: int64

In [28]:
df['user_id'].value_counts()[:10]

C:\Users\user\AppData\Local\Temp\ipykernel_11276\1004146746.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df['user_id'].value_counts()[:10]


20450    185
10843    168
2688     160
22576    135
2893     128
9023      96
48        91
21671     90
9751      79
23232     75
Name: user_id, dtype: int64

In [29]:
df_1511 = df.copy()

remove_list = [20450,10843,2688,22576,2893]
remove_1 = df_1511[df_1511['user_id']==remove_list[0]].index
remove_2 = df_1511[df_1511['user_id']==remove_list[1]].index
remove_3 = df_1511[df_1511['user_id']==remove_list[2]].index
remove_4 = df_1511[df_1511['user_id']==remove_list[3]].index
remove_5 = df_1511[df_1511['user_id']==remove_list[4]].index
df_1511.drop(remove_1,inplace=True)
df_1511.drop(remove_2,inplace=True)
df_1511.drop(remove_3,inplace=True)
df_1511.drop(remove_4,inplace=True)
df_1511.drop(remove_5,inplace=True)
len(df_1511)

33600

# Recommender based on rating - This is the start point!

In [30]:
df_cp = df_1511.copy()

df_cp = df_cp[['genres', 'original_title',
       'overview', 'runtime', 'vote_average', 'username', 'rating', 'total',
       'date', 'review', 'cast', 'crew', 'user_id', 'movie_id']]

in_use = df_cp[['user_id','movie_id','rating']]
in_use.rating.replace({-1:np.nan},regex=True, inplace=True)

piv = in_use.pivot_table(index=['user_id'],columns=['movie_id'],values='rating')

piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)),axis=1)
piv_norm.index
# piv_norm

import scipy as sp
piv_norm.fillna(0,inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:,(piv_norm!=0).any(axis=0)]

piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

item_sim_df = pd.DataFrame(item_similarity, index = piv_norm.index, columns=piv_norm.index)
user_sim_df = pd.DataFrame(user_similarity, index = piv_norm.columns,columns=piv_norm.columns)

C:\Users\user\AppData\Local\Temp\ipykernel_11276\1472195348.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_use.rating.replace({-1:np.nan},regex=True, inplace=True)


In [31]:
def top_movies(movie_id):
  cnt = 1
  print("Similar movies to {}:\n".format(movie_id))
  result = item_sim_df.loc[~item_sim_df.index.isin([movie_id]),movie_id].sort_values(ascending=False)[:10]

  for id, score in result.items():
    print('No. {}: {}({:.2f})'.format(cnt,id,score))
    cnt+=1

top_movies(245)

Similar movies to 245:

No. 1: 321(0.10)
No. 2: 262(0.08)
No. 3: 525(0.06)
No. 4: 208(0.06)
No. 5: 131(0.03)
No. 6: 251(0.03)
No. 7: 396(0.03)
No. 8: 330(0.02)
No. 9: 607(0.01)
No. 10: 123(0.01)


C:\Users\user\AppData\Local\Temp\ipykernel_11276\2715913229.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  result = item_sim_df.loc[~item_sim_df.index.isin([movie_id]),movie_id].sort_values(ascending=False)[:10]


In [32]:
user_sim_df

user_id,20,24,45,46,48,61,66,67,76,82,...,23159,23173,23174,23176,23178,23179,23200,23203,23216,23232
user_id,,,,,,,,,,,,,,,,,,,,,
20,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000
24,0.0,1.000000,0.000000,0.0,0.021077,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.045566,-0.04572,0.000000,0.065124
45,0.0,0.000000,1.000000,0.0,0.004307,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,-0.010401,0.00000,-0.024352,0.045609
46,0.0,0.000000,0.000000,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000
48,0.0,0.021077,0.004307,0.0,1.000000,0.0,0.085553,0.0,0.011340,0.012464,...,0.000000,0.0,0.0,-0.018815,0.0,0.000000,-0.012901,0.00000,-0.005156,0.089038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23179,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,1.000000,0.000000,0.00000,0.000000,0.021163
23200,0.0,0.045566,-0.010401,0.0,-0.012901,0.0,0.000000,0.0,0.071798,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,1.000000,0.00000,0.017730,0.072119
23203,0.0,-0.045720,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.022758,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,1.00000,0.000000,0.000000


In [33]:
import operator
def similar_user_recom(user):
  if user not in piv_norm.columns:
    return('No data available on user {}'.format(user))
  
  # get 10 users by the highest similarity
  sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:101]
  best = []
  most_common={}

  for i in sim_users:
    # Get the list of items that 10 users with high similarity gave high ratings.
    # However, be careful that the item you are recommending must be an item that has not been evaluated by the target user.
    result_sorted = piv_norm.loc[:,i][(piv_norm.loc[:,user]==0)].sort_values(ascending=False)
    best.append(result_sorted.index[:5].tolist())
  for j in range(len(best)):
    for k in best[j]:
      if k in most_common:
        most_common[k]+=1
      else:
        most_common[k]=1
  sorted_list = sorted(most_common.items(),key=operator.itemgetter(1),reverse=True)
  return sorted_list[:5]

similar_user_recom(48)

[(499, 69), (1, 64), (500, 56), (498, 50), (496, 30)]

In [34]:
# movie_list = []

# for i in range(493,501):
#   movie_list.append(i)

# for i in movie_list:
#   print(df_1511.loc[df_1511['movie_id']==i]['original_title'])

In [35]:
# df_1511.loc[df_1511['movie_id']==1]['original_title']

In [36]:
sentiment = pd.read_pickle('D:/Hyunho Shin - DTU/Autumn-2022/Computational Tools for Data Science/project/recommenderrrrr/sentiment_dataset_v2.pkl')
sentiment

,original_title,sentiment_per_user,average_sentiment
0,10 Cloverfield Lane,"{'DVR_Brale': 5.3676056338028175, 'harshjat-76...",5.393406
1,10 Things I Hate About You,"{'bevo-13678': 5.210909090909091, 'Julia-32': ...",5.550725
2,12 Angry Men,"{'sngweiyang': 5.276160000000004, 'thor2029': ...",5.467997
3,12 Years a Slave,"{'jeremycorbyn': 5.375270270270271, 'Pigeon_do...",5.360770
4,127 Hours,"{'mksandhu52': 5.88121739130435, 'coreyjdenfor...",5.416480
...,...,...,...
817,Yes Man,"{'Leofwine_draca': 5.2854929577464835, 'tjdimo...",5.525409
818,Zero Dark Thirty,"{'grantss': 5.4443181818181845, 'Baker8712': 4...",5.356474
819,Zodiac,"{'DamianM88': 5.561980198019805, 'film_riot': ...",5.355870
820,Zombieland,"{'mohit_sinsniwal': 5.3754444444444465, 'Cinea...",5.413930


In [37]:
# df_sent = pd.merge(sentiment,df_1511,how='inner',on='original_title').reset_index()
# df_sent

sent = pd.DataFrame(columns=['original_title','username','sentiment'])
sent

,original_title,username,sentiment


In [38]:
cnt = 0
for i in range(len(sentiment)):
    tmp = sentiment.iloc[i]
    name = tmp['original_title']
    dictionary = tmp['sentiment_per_user']
    for key,value in dictionary.items():
        # hi = sent.iloc[cnt]
        # hi['original_title'] = name
        # hi['user'] = key
        # hi['sentiment'] = value
        row = []
        row.append(name)
        row.append(key)
        row.append(value)
        sent = sent.append(pd.Series(row,index=sent.columns),ignore_index=True)
        # cnt+=1
        # print(1)
    # for j in range(len(dictionary)):
        
sent

C:\Users\user\AppData\Local\Temp\ipykernel_11276\3864537.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent = sent.append(pd.Series(row,index=sent.columns),ignore_index=True)


,original_title,username,sentiment
0,10 Cloverfield Lane,DVR_Brale,5.367606
1,10 Cloverfield Lane,harshjat-76286,5.632857
2,10 Cloverfield Lane,annoyingcashier,5.427778
3,10 Cloverfield Lane,Reno-Rangan,5.368113
4,10 Cloverfield Lane,siderite,5.334291
...,...,...,...
41095,Zulu,emguy,5.352903
41096,Zulu,wendycooke107,5.238364
41097,Zulu,Natheer,5.380189
41098,Zulu,Enchorde,5.244913


In [39]:
df_1511.columns

Index(['Unnamed: 0.1', 'index', 'Unnamed: 0', 'genres', 'original_title',
       'overview', 'runtime', 'vote_average', 'username', 'rating', 'total',
       'date', 'review', 'cast', 'crew', 'user_id', 'movie_id'],
      dtype='object')

In [40]:
len(df_1511)

33600

In [41]:
sent.columns

Index(['original_title', 'username', 'sentiment'], dtype='object')

In [42]:
df_1511['username'].value_counts()

Quinoa1984            96
hitchcockthelegend    91
Theo Robertson        90
TheLittleSongbird     79
bkoganbing            75
                      ..
halima-shaline         1
curryspice             1
Rocco3000              1
mikael_ronne           1
steve-hosking          1
Name: username, Length: 23243, dtype: int64

In [43]:
df_sent = pd.merge(sent,df_1511,how='inner',on=['original_title','username']).reset_index()
# df_sent

In [44]:
df_sent_cp = df_sent.copy()
df_sent_cp = df_sent_cp.dropna()

In [45]:
df_sent_cp['sentiment'].value_counts

<bound method IndexOpsMixin.value_counts of 0       5.367606
1       5.313991
2       5.369028
3       5.742762
4       5.350783
          ...   
3434    5.257235
3435    5.443273
3436    5.293197
3437    5.342932
3438    5.238364
Name: sentiment, Length: 3439, dtype: float64>

# Recommender based on sentiment score

In [46]:
sent_cp = df_sent_cp.copy()

in_use = sent_cp[['user_id','movie_id','sentiment']]
in_use.sentiment.replace({-1:np.nan},regex=True, inplace=True)

piv = in_use.pivot_table(index=['user_id'],columns=['movie_id'],values='sentiment')

piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)),axis=1)
piv_norm.index
# piv_norm

import scipy as sp
piv_norm.fillna(0,inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:,(piv_norm!=0).any(axis=0)]

piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

item_sim_df = pd.DataFrame(item_similarity, index = piv_norm.index, columns=piv_norm.index)
user_sim_df = pd.DataFrame(user_similarity, index = piv_norm.columns,columns=piv_norm.columns)

C:\Users\user\AppData\Local\Temp\ipykernel_11276\962549627.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_use.sentiment.replace({-1:np.nan},regex=True, inplace=True)


In [47]:
user_sim_df

user_id,48,76,117,141,202,312,762,769,814,827,...,22494,22818,22864,22925,23049,23053,23056,23121,23216,23232
user_id,,,,,,,,,,,,,,,,,,,,,
48,1.000000,0.000000,0.022744,0.0,0.0,0.027042,0.0,0.000000,0.0,0.093164,...,0.0,0.130952,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
76,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.0,0.018223,0.0,0.000000,...,0.0,0.000000,0.547138,0.0,0.0,0.000000,0.413554,0.0,0.0,0.548415
117,0.022744,0.000000,1.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
141,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
202,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23053,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.025967
23056,0.000000,0.413554,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.0,0.000000
23121,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.0,0.0,0.000000


In [48]:
import operator
def similar_user_recom(user):
  if user not in piv_norm.columns:
    return('No data available on user {}'.format(user))
  
  # get 10 users by the highest similarity
  sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:101]
  best = []
  most_common={}

  for i in sim_users:
    # Get the list of items that 10 users with high similarity gave high ratings.
    # However, be careful that the item you are recommending must be an item that has not been evaluated by the target user.
    result_sorted = piv_norm.loc[:,i][(piv_norm.loc[:,user]==0)].sort_values(ascending=False)
    best.append(result_sorted.index[:5].tolist())
  for j in range(len(best)):
    for k in best[j]:
      if k in most_common:
        most_common[k]+=1
      else:
        most_common[k]=1
  sorted_list = sorted(most_common.items(),key=operator.itemgetter(1),reverse=True)
  # print(result_sorted)
  return sorted_list[:5]

similar_user_recom(48)

[(1, 82), (491, 78), (490, 69), (493, 29), (500, 26)]

In [49]:
user_sim_df[48].sort_values(ascending=False)

user_id
48       1.000000
1749     0.362871
14195    0.316486
9570     0.297091
7275     0.292345
           ...   
9489    -0.002923
8564    -0.025980
18295   -0.082991
16427   -0.131921
9114    -0.292345
Name: 48, Length: 240, dtype: float64